In [ ]:
# Импорты
import os
import random

import cv2
import numpy as np
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# 0. Конфигурация

DATA_DIR = r"C:\Users\N1lr\PycharmProjects\IP\lab_9\data"
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

SYMBOLS = sorted(
    [
        d
        for d in os.listdir(DATA_DIR)
        if os.path.isdir(os.path.join(DATA_DIR, d))
    ]
)

print("Найдены классы (папки):")
print(SYMBOLS)


Найдены классы (папки):
['(', ')', '+', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'X', 'h', 't', 'times', 'w', 'y']

In [ ]:
# 1. Вспомогательные функции

def load_gray_image(path, target_size=None):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    if target_size is not None:
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    return img


def put_on_canvas(img, canvas_size=45):
    h, w = img.shape
    canvas = np.zeros((canvas_size, canvas_size), dtype=np.uint8)
    if img.max() > 0:
        img = (img.astype(np.float32) / img.max() * 255).astype(np.uint8)

    y_offset = (canvas_size - h) // 2
    x_offset = (canvas_size - w) // 2
    canvas[y_offset:y_offset + h, x_offset:x_offset + w] = img
    return canvas


def show_images_row(images, titles=None, figsize=(12, 3), cmap="gray"):
    if len(images) == 0:
        print("Пустой список изображений")
        return

    plt.figure(figsize=figsize)
    for idx, img in enumerate(images, 1):
        plt.subplot(1, len(images), idx)
        plt.imshow(img, cmap=cmap)
        if titles is not None:
            plt.title(titles[idx - 1])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
# 2. Загрузка датасета для обучения k-NN


def load_dataset(data_dir, symbols, train_part=0.8, target_size=45):
    X = []
    y = []

    for label in symbols:
        folder = os.path.join(data_dir, label)
        files = [
            f for f in os.listdir(folder)
            if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"))
        ]
        print(f"Символ '{label}': найдено {len(files)} изображений")

        for fname in files:
            path = os.path.join(folder, fname)
            img = load_gray_image(path)
            if img is None:
                continue

            # чуть-чуть нормализации: бинаризация и центрирование в квадрат
            _, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            img_resized = cv2.resize(img_bin, (target_size, target_size), interpolation=cv2.INTER_AREA)
            canvas = put_on_canvas(img_resized, canvas_size=target_size)

            X.append(canvas.flatten())
            y.append(label)

    X = np.array(X, dtype=np.float32)
    y = np.array(y)

    print(f"Всего изображений: {len(y)}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_part, random_state=RANDOM_SEED, stratify=y
    )

    print(f"Тренировочных: {len(y_train)}, тестовых (для проверки k): {len(y_test)}")

    return X_train, X_test, y_train, y_test

X_train, X_val, y_train, y_val = load_dataset(DATA_DIR, SYMBOLS, train_part=0.85, target_size=45)

Символ '(': найдено 14294 изображений

Символ ')': найдено 14355 изображений

Символ '+': найдено 25112 изображений

Символ ',': найдено 1906 изображений

Символ '-': найдено 33997 изображений

Символ '0': найдено 6914 изображений

Символ '1': найдено 26520 изображений

Символ '2': найдено 26141 изображений

Символ '3': найдено 10909 изображений

Символ '4': найдено 7396 изображений

Символ '5': найдено 3545 изображений

Символ '6': найдено 3118 изображений

Символ '7': найдено 2909 изображений

Символ '8': найдено 3068 изображений

Символ '9': найдено 628 изображений

Символ 'X': найдено 26594 изображений

Символ 'h': найдено 1464 изображений

Символ 't': найдено 3274 изображений

Символ 'times': найдено 3251 изображений

Символ 'w': найдено 556 изображений

Символ 'y': найдено 9340 изображений

Всего изображений: 225291

Тренировочных: 191497, тестовых (для проверки k): 33794

In [ ]:
# ============================================================
# 3. Генерация последовательности символов
# ============================================================

class SequenceBuilder:
    def __init__(self, data_dir, symbols):
        self.data_dir = data_dir
        self.symbols = symbols
        self.samples = {}
        self._prepare_samples()

    def _prepare_samples(self):
        """Для каждого символа загружаем список путей, чтобы потом быстро выбирать случайные изображения."""
        for label in self.symbols:
            folder = os.path.join(self.data_dir, label)
            files = [
                os.path.join(folder, f)
                for f in os.listdir(folder)
                if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"))
            ]
            if files:
                self.samples[label] = files

        print("Символы, пригодные для генерации:")
        print(sorted(self.samples.keys()))

    def generate_sequence(self, length=8, spacing=15):
        """Генерируем одну последовательность символов как одно изображение."""
        chosen_labels = [random.choice(list(self.samples.keys())) for _ in range(length)]
        chosen_paths = [random.choice(self.samples[label]) for label in chosen_labels]

        imgs = []
        for path in chosen_paths:
            img = load_gray_image(path)
            if img is None:
                continue
            _, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            imgs.append(img_bin)

        if not imgs:
            return None

        heights = [im.shape[0] for im in imgs]
        max_h = max(heights)
        processed = []
        for im in imgs:
            h, w = im.shape
            canvas = np.zeros((max_h, w), dtype=np.uint8)
            y0 = (max_h - h) // 2
            canvas[y0:y0 + h, :] = im
            processed.append(canvas)

        total_width = sum(im.shape[1] for im in processed) + spacing * (len(processed) - 1)
        sequence_img = np.zeros((max_h, total_width), dtype=np.uint8)

        x = 0
        for im in processed:
            h, w = im.shape
            sequence_img[:, x:x + w] = im
            x += w + spacing

        text = "".join(chosen_labels)
        print(f"Сгенерирована последовательность: {text}")

        return {
            "image": sequence_img,
            "labels": chosen_labels,
            "text": text,
        }


seq_builder = SequenceBuilder(DATA_DIR, SYMBOLS)
sequence_data = seq_builder.generate_sequence(length=10, spacing=20)

plt.figure(figsize=(10, 3))
plt.imshow(sequence_data["image"], cmap="gray")
plt.title(f"Сгенерированная последовательность: {sequence_data['text']}")
plt.axis("off")
plt.show()

Символы, пригодные для генерации:

['(', ')', '+', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'X', 'h', 't', 'times', 'w', 'y']

Сгенерирована последовательность: y,(322-,t+

![Изображение](Figure_1.png)

In [ ]:
# 4. Сегментация последовательности на отдельные символы

class SequenceSegmenter:
    def __init__(self, min_width=10):
        self.min_width = min_width

    def segment(self, seq_img):
        """На входе бинарное изображение с белыми символами на чёрном фоне. Возвращает список вырезанных, центрированных символов."""
        if seq_img.mean() > 127:
            _, binary = cv2.threshold(seq_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        else:
            _, binary = cv2.threshold(seq_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        kernel = np.ones((3, 3), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)

        plt.figure(figsize=(8, 3))
        plt.imshow(binary, cmap="gray")
        plt.title("Бинарное изображение для сегментации")
        plt.axis("off")
        plt.show()

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

        crops = []
        boxes = []

        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w < self.min_width:
                continue
            pad = 2
            x0 = max(x - pad, 0)
            y0 = max(y - pad, 0)
            x1 = min(x + w + pad, binary.shape[1])
            y1 = min(y + h + pad, binary.shape[0])

            crop = binary[y0:y1, x0:x1]
            crops.append(crop)
            boxes.append((x0, y0, x1, y1))

        print(f"Найдено символов по контурам: {len(crops)}")

        vis = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
        for (x0, y0, x1, y1) in boxes:
            cv2.rectangle(vis, (x0, y0), (x1, y1), (0, 255, 0), 1)

        plt.figure(figsize=(8, 3))
        plt.imshow(vis)
        plt.title("Обнаруженные контуры символов")
        plt.axis("off")
        plt.show()

        normalized = []
        for crop in crops:
            crop_resized = cv2.resize(crop, (45, 45), interpolation=cv2.INTER_AREA)
            canvas = put_on_canvas(crop_resized, canvas_size=45)
            normalized.append(canvas)

        if normalized:
            show_images_row(normalized, figsize=(min(15, 3 * len(normalized)), 3))

        return normalized


segmenter = SequenceSegmenter()
cropped_symbols = segmenter.segment(sequence_data["image"])
print(f"Фактически извлечено символов: {len(cropped_symbols)}")



![Изображение](Figure_2.png)

Найдено символов по контурам: 9


![Изображение](Figure_3.png)

Фактически извлечено символов: 9

In [ ]:
# 5. Обучение k-NN моделей

def train_knn_models(X_train, y_train, k_values=(1, 3, 5, 7)):
    models = {}
    for k in k_values:
        print(f"Обучаем k-NN с k={k} ...")
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train, y_train)
        models[f"knn{k}"] = model
    print("Все модели обучены.")
    return models


models = train_knn_models(X_train, y_train, k_values=(1, 3, 5, 7))

for name, model in models.items():
    acc = model.score(X_val, y_val)
    print(f"{name}: accuracy на валидационном наборе = {acc:.3f}")


Обучаем k-NN с k=1 ...

Обучаем k-NN с k=3 ...

Обучаем k-NN с k=5 ...

Обучаем k-NN с k=7 ...

Все модели обучены.

knn1: accuracy на валидационном наборе = 0.993

knn3: accuracy на валидационном наборе = 0.937

knn5: accuracy на валидационном наборе = 0.791

knn7: accuracy на валидационном наборе = 0.738

In [ ]:
# 6. Распознавание символов в сгенерированной последовательности

def recognize_sequence(models, cropped_imgs, true_labels):
    true_text = "".join(true_labels)
    print(f"Исходная строка (по генератору): {true_text}")

    results = {}

    for name, model in models.items():
        print(f"\n--- Модель {name} ---")
        predicted_chars = []

        for idx, img in enumerate(cropped_imgs, 1):
            vec = img.flatten().astype(np.float32)
            pred = model.predict([vec])[0]
            predicted_chars.append(pred)

            marker = "+" if pred == true_labels[idx - 1] else "-"
            print(
                f" {marker} Символ {idx}: истина '{true_labels[idx - 1]}', "
                f"предсказано '{pred}'"
            )

        predicted_text = "".join(predicted_chars)
        correct = sum(t == p for t, p in zip(true_labels, predicted_chars))
        total = len(true_labels)
        acc = correct / total if total > 0 else 0.0

        print(f"Оригинал:    {true_text}")
        print(f"Предсказано: {predicted_text}")
        print(f"Точность: {acc * 100:.1f}% ({correct}/{total})")

        results[name] = {
            "predicted": predicted_text,
            "accuracy": acc,
            "correct": correct,
            "total": total,
        }

    return results


min_len = min(len(sequence_data["labels"]), len(cropped_symbols))
trimmed_labels = sequence_data["labels"][:min_len]
trimmed_imgs = cropped_symbols[:min_len]

recognition_results = recognize_sequence(models, trimmed_imgs, trimmed_labels)

Исходная строка (по генератору): y,(322-,t

--- Модель knn1 ---
 - Символ 1: истина 'y', предсказано 'times'
 - Символ 2: истина ',', предсказано '('
 - Символ 3: истина '(', предсказано '2'
 - Символ 4: истина '3', предсказано ')'
 + Символ 5: истина '2', предсказано '2'
 - Символ 6: истина '2', предсказано '-'
 - Символ 7: истина '-', предсказано ','
 - Символ 8: истина ',', предсказано 't'
 - Символ 9: истина 't', предсказано '+'

Оригинал:    y,(322-,t

Предсказано: times(2)2-,t+

Точность: 11.1% (1/9)


--- Модель knn3 ---
 - Символ 1: истина 'y', предсказано 'times'
 - Символ 2: истина ',', предсказано '('
 - Символ 3: истина '(', предсказано '2'
 - Символ 4: истина '3', предсказано ')'
 + Символ 5: истина '2', предсказано '2'
 - Символ 6: истина '2', предсказано '-'
 - Символ 7: истина '-', предсказано ','
 - Символ 8: истина ',', предсказано 't'
 - Символ 9: истина 't', предсказано '+'

Оригинал:    y,(322-,t

Предсказано: times(2)2-,t+

Точность: 11.1% (1/9)


--- Модель knn5 ---
 - Символ 1: истина 'y', предсказано 'times'
 - Символ 2: истина ',', предсказано '('
 - Символ 3: истина '(', предсказано '2'
 - Символ 4: истина '3', предсказано ')'
 + Символ 5: истина '2', предсказано '2'
 - Символ 6: истина '2', предсказано '-'
 - Символ 7: истина '-', предсказано ','
 - Символ 8: истина ',', предсказано 't'
 - Символ 9: истина 't', предсказано '+'

Оригинал:    y,(322-,t

Предсказано: times(2)2-,t+

Точность: 11.1% (1/9)


--- Модель knn7 ---
 - Символ 1: истина 'y', предсказано 'times'
 - Символ 2: истина ',', предсказано '('
 - Символ 3: истина '(', предсказано '2'
 - Символ 4: истина '3', предсказано ')'
 + Символ 5: истина '2', предсказано '2'
 - Символ 6: истина '2', предсказано '-'
 - Символ 7: истина '-', предсказано ','
 - Символ 8: истина ',', предсказано '-'
 - Символ 9: истина 't', предсказано '+'

Оригинал:    y,(322-,t

Предсказано: times(2)2-,-+

Точность: 11.1% (1/9)


In [ ]:
# 7. Финальный вывод по моделям

print("ФИНАЛЬНОЕ СРАВНЕНИЕ МОДЕЛЕЙ")

sorted_res = sorted(
    recognition_results.items(),
    key=lambda x: x[1]["accuracy"],
    reverse=True,
)

for name, res in sorted_res:
    print(f"{name}: {res['accuracy']:.1%} - '{res['predicted']}'")

best_acc = sorted_res[0][1]["accuracy"]
best_models = [name for name, res in recognition_results.items() if res["accuracy"] == best_acc]

if len(best_models) == 1:
    print(f"ЛУЧШАЯ МОДЕЛЬ: {best_models[0]} с точностью {best_acc:.1%}")
else:
    print(f"ЛУЧШИЕ МОДЕЛИ с точностью {best_acc:.1%}: {', '.join(best_models)}")

ФИНАЛЬНОЕ СРАВНЕНИЕ МОДЕЛЕЙ

knn1: 11.1% - 'times(2)2-,t+'

knn3: 11.1% - 'times(2)2-,t+'

knn5: 11.1% - 'times(2)2-,t+'

knn7: 11.1% - 'times(2)2-,-+'

ЛУЧШИЕ МОДЕЛИ с точностью 11.1%: knn1, knn3, knn5, knn7